Download modelo

In [1]:
!wget -O model_last.safetensors "https://huggingface.co/Tharyck/audios-multispeaker-refine/resolve/main/model_last.safetensors"
!wget -O vocab.txt "https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/vocab.txt"

--2025-05-13 00:47:24--  https://huggingface.co/Tharyck/audios-multispeaker-refine/resolve/main/model_last.safetensors
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /Tharyck/multispeaker-ptbr-f5tts/resolve/main/model_last.safetensors [following]
--2025-05-13 00:47:24--  https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/model_last.safetensors
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/9f/c8/9fc843f07f25545540816d5d615555591a0e2a404bb3c24c1f965f5d098acd1b/39435f0d0179abc166cd443c6bc60b8a55f7908e368b58887853e4a7d94d488b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model_last.safetensors%3B+filename%3D%22model_last.safetensors%22%3B&Expires=1747100845&Policy=eyJ

Configuraçoes para o modelo

In [2]:
!mkdir configs
!mkdir audio_ref
!touch input.txt
!echo "Este é um teste de referência." > input.txt
!wget -O configs/F5TTS_v1_Base.yaml "https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/configs/F5TTS_v1_Base.yaml"
!wget -O audio_ref/F034-0960.wav https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/audio_ref/F034-0960.wav
!wget -O audio_ref/F034-0960.txt https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/audio_ref/F034-0960.txt

--2025-05-13 00:47:53--  https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/configs/F5TTS_v1_Base.yaml
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1948 (1.9K) [text/plain]
Saving to: ‘configs/F5TTS_v1_Base.yaml’

configs/F5TTS_v1_Ba 100%[===================>]   1.90K  --.-KB/s    in 0s      

2025-05-13 00:47:53 (1.23 GB/s) - ‘configs/F5TTS_v1_Base.yaml’ saved [1948/1948]

--2025-05-13 00:47:53--  https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/audio_ref/F034-0960.wav
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/9f/c8/9fc843f07f25545540816d5d615555591a0e2a

In [3]:
  !apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [6]:
!pip install f5-tts==1.1.3


Reason for being yanked: Buggy for custom model usage
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: f5-tts
    Found existing installation: f5-tts 1.1.4
    Uninstalling f5-tts-1.1.4:
      Successfully uninstalled f5-tts-1.1.4


In [2]:
!pip install --upgrade --force-reinstall numpy==1.26.4 torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.0
    Uninstalling torchvision-0.17.0:
      Successfully uninstalled torchvision-0.17.0


Carregar modelo e funçao de inferencia

In [1]:
import os
import subprocess
from f5_tts.infer.utils_infer import (
    infer_process,
    load_model,
    load_vocoder,
    remove_silence_for_generated_wav
)
from omegaconf import OmegaConf
from pathlib import Path
from datetime import datetime
import soundfile as sf
from hydra.utils import get_class
from pydub import AudioSegment

    # Load config & model
config_path = f"configs/F5TTS_v1_Base.yaml"
model_cfg = OmegaConf.load(config_path).model

model_cls = get_class(f"f5_tts.model.{model_cfg.backbone}")
model_arc = model_cfg.arch

ckpt_file="./model_last.safetensors"
vocoder_name="vocos"
model = load_model(
    model_cls, model_arc, ckpt_file, mel_spec_type=vocoder_name, vocab_file="./vocab.txt", device="cuda"
)
vocoder = load_vocoder(vocoder_name=vocoder_name)

def generate_tts(input_text_file, output_audio_file, ref_audio_list, ref_text):
    temp_files = []
    os.makedirs("temp_audio", exist_ok=True)

    with open(input_text_file, "r", encoding="utf-8") as file:
        lines = file.readlines()

    ref_audio_count = len(ref_audio_list)


    for idx, line in enumerate(lines):
        ref_audio = ref_audio_list[idx % ref_audio_count]
        print(ref_audio)
        temp_file = f"temp_audio/temp_{idx}.wav"
        temp_files.append(temp_file)

        print(f"Gerando áudio para: {line.strip()} com referência {ref_audio}")
        audio_segment, final_sample_rate, spectragram = infer_process(
            model_obj=model,
            vocoder=vocoder,
            ref_audio=ref_audio,
            ref_text=ref_text[idx % ref_audio_count],
            gen_text=line.strip(),
            speed=1,
            nfe_step=38,
            device="cuda"
        )

        sf.write(
                temp_file,
                audio_segment,
                final_sample_rate,
            )

        sound = AudioSegment.from_wav(temp_file)
        silence = AudioSegment.silent(duration=300)
        combined = sound + silence

        # Salva novamente o arquivo com o silêncio
        combined.export(temp_file, format="wav")
        # wav = remove_silence_for_generated_wav(wav)
        # sf.write(temp_file, wav, samplerate=24000)

    with open("file_list.txt", "w") as f:
        for temp_file in temp_files:
            f.write(f"file '{temp_file}'\n")

    print("Unindo arquivos de áudio com FFmpeg...")
    subprocess.run(["ffmpeg", "-f", "concat", "-safe", "0", "-i", "file_list.txt", "-af", "highpass=f=200, lowpass=f=3000", "-acodec", "libmp3lame", output_audio_file], check=True)
    print(f"Áudio final salvo em {output_audio_file}")

    for temp_file in temp_files:
        os.remove(temp_file)
    os.remove("file_list.txt")
    os.rmdir("temp_audio")



vocab :  ./vocab.txt
token :  custom
model :  ./model_last.safetensors 

Download Vocos from huggingface charactr/vocos-mel-24khz


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.yaml:   0%|          | 0.00/461 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.4M [00:00<?, ?B/s]

In [ ]:
import os
import shutil

input_text_file = "input.txt"
output_audio_file = "output.mp3"

ref_audio_dir = "audio_ref"

ref_audio_names = [
    # "0dba1.wav", #1
    # "e5696.wav", #2
    "F034-0960.wav", # 3
    # "3038d.wav", #4
    # "e5695.wav",
    # "o1.wav"
]


ref_audio_list = [os.path.join(ref_audio_dir, name) for name in ref_audio_names]

ref_text = []
for audio_file in ref_audio_list:
    text_filename = os.path.splitext(audio_file)[0] + ".txt"
    text_path = os.path.join(text_filename)

    if os.path.exists(text_path):
        with open(text_path, "r", encoding="utf-8") as f:
            ref_text.append(f.read().strip())
    else:
        ref_text.append("")

if os.path.exists("temp_audio"):
    shutil.rmtree("temp_audio")

if os.path.exists(output_audio_file):
    os.remove(output_audio_file)

os.makedirs("temp_audio", exist_ok=True)


generate_tts(input_text_file, output_audio_file, ref_audio_list, ref_text)

from IPython.display import Audio
Audio(output_audio_file)


In [ ]:
from IPython.display import Audio

Audio("output.mp3")